<a href="https://colab.research.google.com/github/mctrinh/ggColab/blob/main/tf2_quick_start_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import TensorFlow into the program.

In [31]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version:  2.7.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [32]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# print(x_train.shape)
# print(x_test.shape)

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")    # add a new axis at the end of the tensor
x_test = x_test[..., tf.newaxis].astype("float32")
# print(x_train.shape)
# print(x_test.shape)

Use `tf.data` to batch and shuffle the dataset:

In [33]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras/sequential_model#model_subclassing):

In [34]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)
  
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training:

In [35]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [36]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [37]:
from numpy.lib.function_base import gradient
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [38]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different 
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [39]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result() * 100}, '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result() * 100}')


Epoch 1, Loss: 0.1401280164718628, Accuracy: 95.77666473388672, Test Loss: 0.05999501794576645, Test Accuracy: 98.0
Epoch 2, Loss: 0.04344845935702324, Accuracy: 98.6199951171875, Test Loss: 0.05069531872868538, Test Accuracy: 98.32999420166016
Epoch 3, Loss: 0.021655118092894554, Accuracy: 99.32500457763672, Test Loss: 0.06815283000469208, Test Accuracy: 97.7699966430664
Epoch 4, Loss: 0.013124204240739346, Accuracy: 99.58666229248047, Test Loss: 0.06450601667165756, Test Accuracy: 98.22999572753906
Epoch 5, Loss: 0.009664596989750862, Accuracy: 99.69666290283203, Test Loss: 0.06547334045171738, Test Accuracy: 98.33999633789062


The image classifier is trained to 98.4% accuracy on this dataset.